In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader,TensorDataset
warnings.filterwarnings('ignore')

In [2]:
# Block1=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Blocks/Block 1.csv')
# Block2=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Blocks/Block 2.csv')

Block=np.vstack((
    pd.read_csv('Blocks/Block 31.csv').drop(['27','23'],axis=1).values,
    pd.read_csv('Blocks/Block 32.csv').drop(['27','23'],axis=1).values
))

test=pd.read_csv('test.csv').drop(['27','23'],axis=1)

In [7]:
X=torch.FloatTensor(Block[:,1:]).cuda('cuda:0')
Y=  torch.LongTensor(Block[:,0]).cuda('cuda:0')

X_test_all=torch.FloatTensor(test.drop('label',axis=1).values).cuda('cuda:0')
Y_test_all=             torch.LongTensor(test['label'].values).cuda('cuda:0')

In [29]:
# Xavier初始化权重

def xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)

class MyNet(nn.Module):
    def __init__(self,input_dim):
        super().__init__()
        self.f1=nn.Linear(input_dim,input_dim*6)
        self.drop1=nn.Dropout(p=0.2)
        self.s1=nn.ReLU()
        
        self.f2=nn.Linear(input_dim*6,input_dim*6)
        self.drop2=nn.Dropout(p=0.3)
        self.s2=nn.ReLU()
        
        self.f3=nn.Linear(input_dim*6,input_dim*6)
        self.drop3=nn.Dropout(p=0.5)
        self.s3=nn.ReLU()    
        
        self.f4=nn.Linear(input_dim*6,input_dim*6)
        self.drop4=nn.Dropout(p=0.3)
        self.s4=nn.ReLU()
        
        self.f5=nn.Linear(input_dim*6,input_dim*6)
        self.drop5=nn.Dropout(p=0.2)
        self.s5=nn.ReLU()
            
        self.f6=nn.Linear(input_dim*6,2)   
        self.s6=nn.Softmax(dim=1)
        
    
    def forward(self,x):
        x=self.s1(self.drop1(self.f1(x)))
        x=self.s2(self.drop2(self.f2(x)))
        x=self.s3(self.drop3(self.f3(x)))
        x=self.s4(self.drop4(self.f4(x)))
        x=self.s5(self.drop5(self.f5(x)))
        return self.s6(self.f6(x))
        
#     预测
    def predict(self,x):
        with torch.no_grad():
            x=self.s1(self.f1(x))
            x=self.s2(self.f2(x))
            x=self.s3(self.f3(x))
            x=self.s4(self.f4(x))
            x=self.s5(self.f5(x))
            p=self.s6(self.f6(x))
            return (p[:,0]<p[:,1])
    
#     准确率 
    def acc(self,x,y):
        with torch.no_grad():
            return float(((self.predict(x)==y).sum()/len(y)))

In [30]:
net=MyNet(X.shape[1]).cuda("cuda:0")
net.apply(xavier);

net.load_state_dict(torch.load('Params/NN16666去0.7相关_23+27/Net 3300.params'))

<All keys matched successfully>

In [13]:
# 用于绘制曲线
losses=[]
train_acc=[]
test_acc=[]

# 用于随机抽测试集
idx=list(range(len(test)))
n_train=len(X)

In [32]:
optimizer=torch.optim.Adam(net.parameters(), lr=0.0125, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.000001, amsgrad=False)
criterion=torch.nn.CrossEntropyLoss()
epochs=100


for i in range(epochs):
    
    y_pred=net.forward(X)
    loss=criterion(y_pred,Y)
    

    if(i%100)==0:
        losses.append(float(loss))
        train_acc.append( net.acc(X,Y) )
        
        np.random.shuffle(idx)

        X_test=X_test_all[idx[n_train:n_train*2]].cuda('cuda:0')
        Y_test=Y_test_all[idx[n_train:n_train*2]].cuda('cuda:0')
        
        test_acc.append(net.acc(X_test,Y_test))
        
    if(i%(int(epochs*0.1))==0):
        print(str((i/epochs)*100)+'%',end='\t')
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print('100%')
torch.save(net.state_dict(),'Params/NN16666去0.7相关_23+27/Net '+str(len(train_acc)*100)+'.params')


RuntimeError: CUDA out of memory. Tried to allocate 184.00 MiB (GPU 0; 4.00 GiB total capacity; 2.76 GiB already allocated; 0 bytes free; 2.97 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
tr_acc=net.acc(X,Y)
te_acc=net.acc(X_test,Y_test)

%matplotlib inline
fig,ax=plt.subplots(1,2,figsize=(16,6))

ax[0].set_xlabel('x100 epochs')
ax[0].set_title('Loss')
ax[0].plot(   losses,label='Train loss')
ax[1].plot(train_acc,label='Train acc = '+str(tr_acc))
ax[1].plot( test_acc,label='Test acc = '+str(te_acc))
ax[1].set_xlabel('x100 epochs')
ax[1].set_title('Accuracy')
plt.legend()
print('epoch: '+str(len(train_acc*100)))
print('Train acc: ',net.acc(X,Y))
print('Test acc:  ',net.acc(X_test,Y_test))
plt.savefig('Params/NN16666去0.7相关_23+27/Net'+str(len(train_acc)*100)+'.png')
